# 예보 데이터 불러오기
[인증키 확인](https://data.go.kr/iim/api/selectAPIAcountView.do)

In [ ]:
! pip install xmltodict

In [1]:
import pandas as pd
from urllib.parse import urlencode, quote_plus
from urllib.request import urlopen
import json
import xmltodict

In [2]:
# API
# ServiceKey = input('encoded api key를 입력해주세요 > ')
key = 'your key'

In [19]:
# 충남 당진시 석문면 : (53, 144)
# 울산광역시 남구 선암동 : (102, 83)
n_row = 113
base_date = 20210701
base_time = '0300'
x = 102
y = 83

url_base = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
queryParams = urlencode({
    # quote_plus('ServiceKey'): key,
    quote_plus('numOfRows'): n_row,      # 불러오고 싶은만큼 가져오기
    quote_plus('pageNo'): 1,
    quote_plus('dataType'): 'XML',
    quote_plus('base_date'): base_date,  # 예보 발표 날짜
    quote_plus('base_time'): base_time,  # 예보 발표 시간
    quote_plus('nx'): str(x),
    quote_plus('ny'): str(y),
})

url = f'{url_base}?serviceKey={key}&{queryParams}'
response = urlopen(url)
results = response.read().decode('utf-8')
results_json = xmltodict.parse(results)

data = json.loads(json.dumps(results_json))

# print(data)

logs = data['response']['body']['items']['item']
df = pd.DataFrame(logs)

df
# for log in logs:
#     s_tmp = pd.Series(log)
#     df_tmp = pd.DataFrame(s_tmp)
#     df = pd.concat([df, df_tmp.T])

# def fcst_time(time):
#     temp = f'{time[:4]}-{time[4:6]}-{time[6:8]} {time[9:11]}:{time[11:]}'
#     datetime = pd.to_datetime(temp)
#     return datetime

# df['time'] = df[['fcstDate', 'fcstTime']].apply(lambda t: ' '.join(t), axis=1)
# df['time'] = df['time'].apply(fcst_time)

# fcst_df = pd.DataFrame(columns=['time', 'fcst_temp', 'fcst_hum', 'fcst_wspeed',
#                                 'fcst_wdirect', 'fcst_cloud'])

,baseDate,baseTime,category,nx,ny,obsrValue
0,20210701,0300,PTY,102,83,0
1,20210701,0300,REH,102,83,83
2,20210701,0300,RN1,102,83,0
3,20210701,0300,T1H,102,83,21.5
4,20210701,0300,UUU,102,83,1
5,20210701,0300,VEC,102,83,340
6,20210701,0300,VVV,102,83,-2.5
7,20210701,0300,WSD,102,83,2.8


In [20]:
def get_data(base_date, base_time, region, key, n_row=100):
    dict_region = {'dangjin': (53, 144), 'ulsan': (102, 83)}
    
    url_base = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
    queryParams = urlencode({
        # quote_plus('ServiceKey'): key,
        quote_plus('numOfRows'): n_row,      # 불러오고 싶은만큼 가져오기
        quote_plus('dataType'): 'XML',
        quote_plus('base_date'): base_date,  # 예보 발표 날짜
        quote_plus('base_time'): base_time,  # 예보 발표 시간
        quote_plus('nx'): str(x),
        quote_plus('ny'): str(y),
    })

    url = f'{url_base}?serviceKey={key}&{queryParams}'
    response = urlopen(url)
    results = response.read().decode('utf-8')
    results_json = xmltodict.parse(results)

    data = json.loads(json.dumps(results_json))
    
    logs = data['response']['body']['items']['item']
    df = pd.DataFrame(logs)
    
    return df
# col_df = ['Forecast time', 'forecast', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']

In [22]:
df = pd.DataFrame()
for i in range(24):
    t = f'{i:0>2}00'
    d = 20210701
    new_df = get_data(d, t, 'ulsan', key)
    df = pd.concat([df, new_df])
df

,baseDate,baseTime,category,nx,ny,obsrValue
0,20210701,0000,PTY,102,83,0
1,20210701,0000,REH,102,83,85
2,20210701,0000,RN1,102,83,0
3,20210701,0000,T1H,102,83,22
4,20210701,0000,UUU,102,83,0.5
...,...,...,...,...,...,...
3,20210701,2300,T1H,102,83,23.2
4,20210701,2300,UUU,102,83,0
5,20210701,2300,VEC,102,83,2
6,20210701,2300,VVV,102,83,-2.8


In [23]:
df2 = pd.DataFrame()
for i in range(24):
    t = f'{i:0>2}00'
    d = 20210701
    new_df2 = get_data(d, t, 'dangjin', key)
    df2 = pd.concat([df2, new_df2])
df2

,baseDate,baseTime,category,nx,ny,obsrValue
0,20210701,0000,PTY,102,83,0
1,20210701,0000,REH,102,83,85
2,20210701,0000,RN1,102,83,0
3,20210701,0000,T1H,102,83,22
4,20210701,0000,UUU,102,83,0.5
...,...,...,...,...,...,...
3,20210701,2300,T1H,102,83,23.2
4,20210701,2300,UUU,102,83,0
5,20210701,2300,VEC,102,83,2
6,20210701,2300,VVV,102,83,-2.8


In [24]:
df.to_csv('ulsan_0701.csv', index=None)
df2.to_csv('dangjin_0701.csv', index=None)

In [ ]:
def fcst_ulsan(base_date, base_time, key, n_row=113):
    
    # ulsan 좌표
    x = 102
    y = 83

    url_base = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getUltraSrtFcst'
    queryParams = urlencode({
        # quote_plus('ServiceKey'): key,
        quote_plus('numOfRows'): n_row,      # 불러오고 싶은만큼 가져오기
        quote_plus('dataType'): 'XML',
        quote_plus('base_date'): base_date,  # 예보 발표 날짜
        quote_plus('base_time'): base_time,  # 예보 발표 시간
        quote_plus('nx'): str(x),
        quote_plus('ny'): str(y),
    })

    url = f'{url_base}?serviceKey={key}&{queryParams}'
    response = urlopen(url)
    results = response.read().decode('utf-8')
    results_json = xmltodict.parse(results)

    data = json.loads(json.dumps(results_json))

    if 'ERROR' in data['response']['header']['resultMsg']:
        print('Fail.')
    else:
        if ###
        print('Success!')

    # data 대강 확인하기
    print(data)
    
    
    logs = data['response']['body']['items']['item']
    df = pd.DataFrame()

    for log in logs:
        s_tmp = pd.Series(log)
        df_tmp = pd.DataFrame(s_tmp)
        df = pd.concat([df, df_tmp.T])

    def fcst_time(time):
        temp = f'{time[:4]}-{time[4:6]}-{time[6:8]} {time[9:11]}:{time[11:]}'
        datetime = pd.to_datetime(temp)
        return datetime

    df['time'] = df[['fcstDate', 'fcstTime']].apply(lambda t: ' '.join(t), axis=1)
    df['time'] = df['time'].apply(fcst_time)

    fcst_df = pd.DataFrame(columns=['time', 'fcst_temp', 'fcst_hum', 'fcst_wspeed',
                                    'fcst_wdirect', 'fcst_cloud'])
    
    return df

In [ ]:
df = fcst_ulsan('20210608', '1000', key)

In [12]:
# ulsan 좌표
x = 102
y = 83

url_base = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getUltraSrtFcst'
key = ServiceKey
queryParams = urlencode({
    # quote_plus('ServiceKey'): key,
    quote_plus('numOfRows'): '113',      # 불러오고 싶은만큼 가져오기
    quote_plus('dataType'): 'XML',
    quote_plus('base_date'): '20210607', # 예보 발표 날짜
    quote_plus('base_time'): '1000',     # 예보 발표 시간
    quote_plus('nx'): str(x),
    quote_plus('ny'): str(y),
})

url = f'{url_base}?serviceKey={key}&{queryParams}'
response = urlopen(url)
results = response.read().decode('utf-8')
results_json = xmltodict.parse(results)

data = json.loads(json.dumps(results_json))

if 'ERROR' in data['response']['header']['resultMsg']:
    print('Fail.')
else:
    print('Success!')

# data 대강 확인하기
print(data)

HTTPError: HTTP Error 500: Internal Server Error

{'response': {'header': {'resultCode': '03', 'resultMsg': 'NO_DATA'}}}

In [10]:
logs = data['response']['body']['items']['item']
df = pd.DataFrame()

for log in logs:
    s_tmp = pd.Series(log)
    df_tmp = pd.DataFrame(s_tmp)
    df = pd.concat([df, df_tmp.T])

def fcst_time(time):
    temp = f'{time[:4]}-{time[4:6]}-{time[6:8]} {time[9:11]}:{time[11:]}'
    datetime = pd.to_datetime(temp)
    return datetime

df['time'] = df[['fcstDate', 'fcstTime']].apply(lambda t: ' '.join(t), axis=1)
df['time'] = df['time'].apply(fcst_time)

fcst_df = pd.DataFrame(columns=['time', 'fcst_temp', 'fcst_hum', 'fcst_wspeed',
                                'fcst_wdirect', 'fcst_cloud'])

df

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny,time
0,20210608,1030,LGT,20210608,1100,0,102,83,2021-06-08 11:00:00
0,20210608,1030,LGT,20210608,1200,0,102,83,2021-06-08 12:00:00
0,20210608,1030,LGT,20210608,1300,0,102,83,2021-06-08 13:00:00
0,20210608,1030,LGT,20210608,1400,0,102,83,2021-06-08 14:00:00
0,20210608,1030,LGT,20210608,1500,0,102,83,2021-06-08 15:00:00
0,20210608,1030,PTY,20210608,1100,0,102,83,2021-06-08 11:00:00
0,20210608,1030,PTY,20210608,1200,0,102,83,2021-06-08 12:00:00
0,20210608,1030,PTY,20210608,1300,0,102,83,2021-06-08 13:00:00
0,20210608,1030,PTY,20210608,1400,0,102,83,2021-06-08 14:00:00
0,20210608,1030,PTY,20210608,1500,0,102,83,2021-06-08 15:00:00
